In [1]:
import numpy as np # linear algebra
import pandas as pd 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.preprocessing import FunctionTransformer
import xgboost
import pickle

from datetime import datetime


In [2]:
def get_mpg(x):
    x = str(x)
    if x == 'nan':
        return np.nan
    elif len(x) <= 2:
        return float(x)
    else:
        return (float(x.split('-')[0])+float(x.split('-')[1]))/2


# Directorio donde se encuentra el archivo
path_for_file = 'archivos_a_procesar/data_cars_new.csv'
df = pd.read_csv(path_for_file)
fecha_hoy = datetime.now().strftime('%Y-%m-%d')


cat_features = ['manufacturer','transmission','drivetrain','fuel_type']
num_features = ['year','mileage', 'mpg', 'driver_reviews_num', 'seller_rating', 'driver_rating']
dum_variables = ['accidents_or_damage', 'one_owner','personal_use_only']
only_features = dum_variables + cat_features + num_features

dive_train_map = {'Front-wheel Drive':'Front Wheel Driver',
                          'FWD':'Front Wheel Driver',
                          'Front-Wheel Drive':'Front Wheel Driver',
                          'Four-wheel Drive':'Four Wheel Driver',
                          'Four-Wheel Drive':'Four Wheel Driver',
                          'Four Wheel Drive':'Four Wheel Driver',
                          '4WD':'Four Wheel Driver',
                          'All-wheel Drive':'All Wheel Driver',
                          'All-Wheel Drive':'All Wheel Driver',
                          'AWD':'All Wheel Driver',
                          'Rear-wheel Drive':'Rear Wheel Driver',
                          'RWD':'Rear Wheel Driver'}


# Quito los nulos
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)


# Transformaciones necesarias   
for i in dum_variables:
    df[i] = df[i].astype(int)

df['drivetrain'] = df['drivetrain'].map(dive_train_map).fillna(value='Others')

df['mpg'] = df['mpg'].map(get_mpg)


with open('models/xgb_model_v1.pkl', 'rb') as file:
    model = pickle.load(file)


data_prediction = df[only_features].copy()

# Predicciones
predicciones = model.predict(data_prediction)
df_predict = pd.DataFrame({'estimacion_precio':predicciones})

# Guardado archivo
resultado = pd.concat([df[['manufacturer','model','year','transmission','drivetrain']],df_predict], axis=1)
resultado.to_csv(f'resutados_prediciones/predicciones_{fecha_hoy}.csv', index=False)

array([19813.06 , 24674.922, 30887.803, ..., 10572.252, 34371.242,
       11924.544], dtype=float32)

,manufacturer,model,year,transmission,drivetrain,estimacion_precio
0,Hyundai,Sonata Sport,2019,6-Speed Automatic,Front Wheel Driver,19813.060547
1,Buick,Envision Essence,2019,6-Speed Automatic,Front Wheel Driver,24674.921875
2,Volvo,XC40 T5 Momentum,2020,8-Speed Automatic,All Wheel Driver,30887.802734
3,Toyota,Highlander LE,2021,8-Speed Automatic,All Wheel Driver,38674.757812
4,Lexus,ES 350 Luxury,2019,8-Speed Automatic,Front Wheel Driver,36318.953125
...,...,...,...,...,...,...
6460,Porsche,911 Turbo S,2022,8-Speed Automatic with Auto-Shift,All Wheel Driver,68956.468750
6461,Mercedes-Benz,GL-Class GL 550 4MATIC,2009,7-Speed Automatic,All Wheel Driver,22493.990234
6462,Nissan,Pathfinder SL,2014,Automatic CVT,Four Wheel Driver,10572.251953
6463,Honda,Civic Type R Touring,2021,6-Speed Manual,Front Wheel Driver,34371.242188
